This is code to better understand the projected distributional bellman update in numpy.

In [164]:
import numpy as np
import math

def softmax(z):
    assert len(z.shape) == 2
    s = np.max(z, axis=1)
    s = s[:, np.newaxis] # necessary step to do broadcasting
    e_x = np.exp(z - s)
    div = np.sum(e_x, axis=1)
    div = div[:, np.newaxis] # dito
    return e_x / div


In [165]:
num_actions = 4

# Min and Max 
v_max = 10
v_min = -10

# number of atoms, because we are including 0
z = 21

#Defined in section 4.1
delta_z = float((v_max - v_min)) / (z - 1)

# random action vector
q_values_vector = softmax(np.random.rand(num_actions, z))

q_values = np.zeros(num_actions)

# Line 1 of Alg. 1
for action_idx in range(num_actions):
    z_idx = 0
    z_stepper = v_min
    while (z_idx != z):
        q_values[action_idx] += q_values_vector[action_idx][z_idx] * z_stepper
        z_stepper += delta_z
        z_idx += 1
        
#Line 2 of Algorithm 1
optimal_action = np.argmax(q_values)


Compute the Projection

In [167]:

#line 3 of Alg 1 (the init to zeros)
projected_q_value_vector = np.zeros((z))

# reward at time t
r_t = 0.0

# Discount factor
gamma = .95

z_idx = 0
z_stepper = v_min
# Line 4 of Alg 1 (the j loop to N)
while (z_idx != z):
    
    #Line 5 of Algoirhtm 1 (Clipping)
    projected_z = max(min(r_t + gamma * z_stepper, v_max), v_min)
    
    # Line 6 of algorithm 1
    b = (projected_z - v_min) / delta_z
    
    # Line 7 of algoirthm 1
    l = int(math.floor(b))
    u = int(math.ceil(b))
    
    # Lines 8 and 9 of algorithm 1
    transition_prob = q_values_vector[optimal_action][z_idx]
    projected_q_value_vector[l] = projected_q_value_vector[l] + transition_prob * (u - b)
    projected_q_value_vector[u] = projected_q_value_vector[u] + transition_prob * (b - l)
    
    z_stepper += delta_z
    z_idx += 1

    # doesn't equal one because of the min and the max. we are clipping 
print projected_q_value_vector
print q_values_vector[optimal_action]


[ 0.01615983  0.03329078  0.03424343  0.06258222  0.05034519  0.0356659
  0.03399998  0.0379598   0.03759498  0.03453206  0.0032346   0.03788008
  0.07801378  0.07316943  0.04498811  0.05500165  0.07357977  0.07986593
  0.0607817   0.04040215  0.02051784]
[ 0.03231965  0.03114718  0.03371199  0.07553449  0.03415445  0.03389276
  0.03190849  0.0371507   0.03558042  0.03260423  0.05619081  0.03208771
  0.07396762  0.07628616  0.04163099  0.04673327  0.06650566  0.0772166
  0.07418786  0.03615329  0.04103569]
